In [1]:
import init
from uuoskit import uuosapi, config
from uuoskit import wasmcompiler
init.init()
test_account1 = init.test_account1

In [2]:
a = {
    "account": test_account1,
    "permission": "active",
    "parent": "owner",
    "auth": {
        "threshold": 1,
        "keys": [
            {
                "key": "EOS6AjF6hvF7GSuSd4sCgfPKq5uWaXvGM2aQtEUCwmEHygQaqxBSV",
                "weight": 1
            },
        ],
        "accounts": [{"permission":{"actor":test_account1,"permission":"eosio.code"},"weight":1}],
        "waits": []
    }
}
r = uuosapi.push_action('eosio', 'updateauth', a, {test_account1:'active'})
print('done!')

done!


In [3]:
test_account1, config.main_token

('helloworld11', 'EOS')

In [4]:
code = r'''
package main

import "github.com/uuosio/chain"
import "github.com/uuosio/chain/logger"

func main() {
    _, _, action := chain.GetApplyArgs()
    if action == chain.NewName("sayhello") {
        payer := chain.NewName("helloworld11")


        a := chain.Action{}
        a.Account = chain.NewName("eosio.token")
        a.Name = chain.NewName("transfer")
        a.AddPermission(chain.NewName("helloworld11"), chain.ActiveName)

        t := chain.Transfer{}
        t.From = chain.NewName("helloworld11")
        t.To = chain.NewName("eosio")
        // Send 1.0 EOS
        t.Quantity.Amount = 10000;
        t.Quantity.Symbol = chain.NewSymbol("EOS", 4);
        a.Data = t.Pack()

        tx := chain.NewTransaction(1)
        tx.Actions = []chain.Action{a}
        tx.Send(1, false, payer)   
        logger.Println("transaction sent")
    }
}
'''

code, abi = uuosapi.compile(test_account1, code, src_type=2)
uuosapi.deploy_code(test_account1, code, vm_type=0)
print('done!')

done!


In [5]:
uuosapi.get_balance(test_account1),uuosapi.get_balance('eosio')

(100000839.5901, 589987473.6273)

In [6]:
r = uuosapi.push_action(test_account1, 'sayhello', b'hello,world', {test_account1:'active'})
print(r['processed']['action_traces'][0]['console'])
print(r['processed']['elapsed'])

transaction sent

1064


In [7]:
uuosapi.get_balance(test_account1),uuosapi.get_balance('eosio')

(100000838.5901, 589987474.6273)